In [180]:
import numpy as np
import pandas as pd

In [181]:
Player_information = pd.read_csv('Player_information.csv', engine='python')
Player_action = pd.read_csv('Player_action.csv', engine='python')
Group_activity = pd.read_csv('Group_activity.csv', engine='python')
labeled_accounts = pd.read_csv('labeled_accounts.csv', engine='python')

# merge 시키고 labeld_accounts에 있는 것만 뽑아오기

In [44]:
final = pd.merge(Player_information, Player_action, how='outer', on = "actor_account")
final = pd.merge(final, Group_activity, how='outer',  on = "actor_account")

In [45]:
final = pd.merge(final, labeled_accounts, how='inner', on = 'actor_account')
final.fillna(0, inplace=True)
print(final.shape)
final.head()

(8100, 24)


,actor_account,login_count,logout_count,login_day_count,play_time,avg_money,ip_count,max_level,sit_count,exp_get_amout,...,use_portal_count,killed_by_pc,killed_by_npc,teleport_count,reborn_count,question_count,total_party_time,guild_join_count,average_party_time,class
0,20000005,259.0,259.0,1.0,637926.0,124.849465,259.0,0.0,270.0,3.891496e+08,...,0.0,47.0,30.0,376.0,51.0,62.0,8862.0,0.0,1477.000000,0
1,20000016,38.0,37.0,1.0,401885.0,43.509962,38.0,42.0,153.0,8.011943e+07,...,0.0,32.0,75.0,343.0,75.0,5893.0,69458.0,0.0,3655.684211,0
2,20000033,43.0,42.0,1.0,240334.0,-6.613812,43.0,38.0,378.0,1.424854e+08,...,0.0,35.0,53.0,337.0,69.0,97.0,43619.0,0.0,2295.736842,0
3,20000035,25.0,24.0,1.0,1205493.0,0.088531,25.0,0.0,207.0,1.377553e+09,...,0.0,0.0,12.0,44.0,10.0,0.0,0.0,0.0,0.000000,0
4,20000051,110.0,109.0,1.0,457433.0,-32.874878,110.0,0.0,320.0,3.950991e+08,...,1.0,311.0,109.0,568.0,261.0,80.0,40934.0,0.0,2558.375000,0


# 모델 학습

In [219]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
#import xgboost as xgb

ModuleNotFoundError: No module named 'xgboost'

In [220]:
model_data = final.copy()
del model_data['actor_account']
del model_data['max_level']
del model_data['guild_join_count']
x = model_data.iloc[:, :-1]
y = model_data.iloc[:, -1]

In [221]:
model_data.head()

,login_count,logout_count,login_day_count,play_time,avg_money,ip_count,sit_count,exp_get_amout,item_get_count,exp_repair_count,...,abyss,use_portal_count,killed_by_pc,killed_by_npc,teleport_count,reborn_count,question_count,total_party_time,average_party_time,class
0,259.0,259.0,1.0,637926.0,124.849465,259.0,270.0,3.891496e+08,31149.0,0.0,...,0.0,0.0,47.0,30.0,376.0,51.0,62.0,8862.0,1477.000000,0
1,38.0,37.0,1.0,401885.0,43.509962,38.0,153.0,8.011943e+07,61579.0,0.0,...,0.0,0.0,32.0,75.0,343.0,75.0,5893.0,69458.0,3655.684211,0
2,43.0,42.0,1.0,240334.0,-6.613812,43.0,378.0,1.424854e+08,24686.0,0.0,...,0.0,0.0,35.0,53.0,337.0,69.0,97.0,43619.0,2295.736842,0
3,25.0,24.0,1.0,1205493.0,0.088531,25.0,207.0,1.377553e+09,109795.0,0.0,...,0.0,0.0,0.0,12.0,44.0,10.0,0.0,0.0,0.000000,0
4,110.0,109.0,1.0,457433.0,-32.874878,110.0,320.0,3.950991e+08,98368.0,0.0,...,0.0,1.0,311.0,109.0,568.0,261.0,80.0,40934.0,2558.375000,0


In [222]:
print(list(y).count(1))
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

600


## 랜덤 포레스트

In [225]:
from sklearn.ensemble import RandomForestClassifier
#500개의 DT classifier만듬. 각 DT당 최대 16개의 leaf node, n_jobs -> cpu 코어 몇개 쓸지
rnd_clf = RandomForestClassifier(n_estimators=2000, max_leaf_nodes=2, criterion='gini', max_depth = 5)
rnd_clf.fit(x_test, y_test)
y_pred_rf = rnd_clf.predict(x_test)

In [226]:
from sklearn.metrics import accuracy_score, f1_score
print("Accuracy: %.2f" %accuracy_score(y_test, y_pred_rf))
print("F1 score: %.2f" %f1_score(y_test, y_pred_rf))

Accuracy: 0.92
F1 score: 0.00


c:\users\백진우\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


## 로지스틱

In [227]:
from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler(feature_range=(0, 1))
# scaler.fit_transform(x_train)
# x_train = scaler.fit_transform(x_train)

from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE

# 모델설정
sm = SMOTE(ratio='auto', kind='regular')

# train데이터를 넣어 복제함
X_resampled, Y_resampled = sm.fit_sample(x_train,list(y_train))

In [267]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()
voting_clf = VotingClassifier(estimators=[("lr", log_clf),("rf", rnd_clf),("svc", svm_clf)], voting="hard")

#log_clf.fit(x_train, y_train)
log_clf.fit(X_resampled, Y_resampled)
# y_pred = log_clf.predict(X_test)
y_pred = np.where(log_clf.predict_proba(x_test)[:,1]>0.65, 1, 0)

c:\users\백진우\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [268]:
print(list(y_pred).count(1))

354


In [269]:
from sklearn.metrics import accuracy_score, f1_score
print("Accuracy: %.2f" %accuracy_score(y_test, y_pred))
print("F1 score: %.2f" %f1_score(y_test, y_pred))

Accuracy: 0.88
F1 score: 0.48


# csv 파일로 만들기

In [197]:
Player_information.to_csv("Player_information.csv", encoding="utf8")
Player_action.to_csv("Player_action.csv", encoding="utf8")
Group_activity.to_csv("Group_activity.csv", encoding="utf8")